In [1]:
import numpy as np
import pandas as pd
from SSA import SSA
import os
from tqdm.notebook import tqdm

In [2]:
def wape(y, y_hat):
    return np.sum(np.abs(y - y_hat)) / np.sum(np.abs(y)) * 100

In [8]:
def get_wapes(path, country, prices_not_returns, test_size):
    result_dict = dict()
    entries = os.listdir(path)
    entries.remove(".DS_Store")

    pbar = tqdm(entries, desc= f"{country} company", total= len(entries))
    for file in pbar:
        df = pd.read_csv(path + file if path[-1] == r"/" else path + r"/" + file).dropna()
        series = np.array(df.Open) if prices_not_returns else np.array(df.Open.pct_change())[1:]

        x_train, x_test = series[:test_size], series[test_size:]

        N = len(x_train)
        L = 20
        K = N - L + 1
        r = None

        hankelian = SSA.x_to_Hankelian(x_train, L)
        d = np.linalg.matrix_rank(hankelian)

        forecast = np.zeros(len(x_test))

        pbar_forecast = tqdm(range(len(x_test)), f"{file[:-14]}: Forecast stage")
        for i in pbar_forecast:
            x_train_loc = np.concatenate([x_train, x_test[:i]], axis= 0)
            f = SSA.forecast(x_train_loc, K, 1, 18)
            forecast[i] = f

        metrics = wape(x_test, forecast)
        result_dict[file[:-14]] = metrics

        print(f"{file[:-14]}: Test error: {metrics:.2}%")

    return result_dict


In [4]:
us_prices = {
    "path": "../Data/American Companies",
    "country": "US",
    "prices_not_returns": True,
    "test_size": -50
}
us_returns = {
    "path": "../Data/American Companies",
    "country": "US",
    "prices_not_returns": False,
    "test_size": -50
}

result_us_prices = get_wapes(**us_prices)
result_us_returns = get_wapes(**us_returns)

us_result_df = pd.DataFrame({
    "Company": result_us_prices.keys(),
    "WAPE (price)": result_us_prices.values(),
    "WAPE (return)": result_us_returns.values()
})

us_result_df.to_csv("ssa_us.csv", index= False)

US company:   0%|          | 0/15 [00:00<?, ?it/s]

GE: Forecast stage:   0%|          | 0/50 [00:00<?, ?it/s]

GE: Test error: 2.0%


AMZN: Forecast stage:   0%|          | 0/50 [00:00<?, ?it/s]

AMZN: Test error: 4.4%


...

EBAY: Test error: 1e+02%


FORD: Forecast stage:   0%|          | 0/50 [00:00<?, ?it/s]

FORD: Test error: 1.4e+02%


DIS: Forecast stage:   0%|          | 0/50 [00:00<?, ?it/s]

DIS: Test error: 1.3e+02%


In [9]:
ch_prices = {
    "path": "../Data/Chinese Companies",
    "country": "CH",
    "prices_not_returns": True,
    "test_size": -50
}

ch_returns = {
    "path": "../Data/Chinese Companies",
    "country": "CH",
    "prices_not_returns": False,
    "test_size": -50
}

result_ch_prices = get_wapes(**ch_prices)
result_ch_returns = get_wapes(**ch_returns)

ch_result_df = pd.DataFrame({
    "Company": result_ch_prices.keys(),
    "WAPE (price)": result_ch_prices.values(),
    "WAPE (return)": result_ch_returns.values()
})

ch_result_df.to_csv("ssa_ch.csv", index= False)

CH company:   0%|          | 0/15 [00:00<?, ?it/s]

China Shenhua: Forecast stage:   0%|          | 0/50 [00:00<?, ?it/s]

China Shenhua: Test error: 2.1%


China Pacific Insurance: Forecast stage:   0%|          | 0/50 [00:00<?, ?it/s]

China Pacific Insurance: Test error: 2.2%


...

CITIC Securities: Test error: 1.1e+02%


PetroChina: Forecast stage:   0%|          | 0/50 [00:00<?, ?it/s]

PetroChina: Test error: 1.5e+02%


China Merchants Bank: Forecast stage:   0%|          | 0/50 [00:00<?, ?it/s]

China Merchants Bank: Test error: 1.1e+02%
